In [ ]:
# Task 2 – Yield Curve Modeling
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline

# ------------------------------
# a) Choice of government securities
# ------------------------------
maturities = np.array([0.5, 2, 5, 10, 20, 30])           # in years
yields_pct = np.array([4.5, 4.7, 4.9, 5.1, 5.3, 5.4])    # example yields in percent

# ------------------------------
# b) Nelson–Siegel model and fit
# ------------------------------
def nelson_siegel(tau, beta0, beta1, beta2, lambd):
    """Nelson–Siegel functional form for par yield."""
    term1 = (1 - np.exp(-tau / lambd)) / (tau / lambd)
    term2 = term1 - np.exp(-tau / lambd)
    return beta0 + beta1 * term1 + beta2 * term2

# Reasonable starting values help convergence; units match 'yields_pct'
p0 = [np.mean(yields_pct), -0.5, 0.5, 2.0]
params, _ = curve_fit(nelson_siegel, maturities, yields_pct, p0=p0, maxfev=10000)
beta0, beta1, beta2, lambd = params

tau_fit = np.linspace(maturities.min(), maturities.max(), 400)
ns_fit = nelson_siegel(tau_fit, *params)

# ------------------------------
# c) Cubic Spline interpolation
# ------------------------------
cs = CubicSpline(maturities, yields_pct, bc_type='not-a-knot')
spline_fit = cs(tau_fit)

# ------------------------------
# d) Plot comparison
# ------------------------------
plt.figure(figsize=(9,6))
plt.scatter(maturities, yields_pct, label="Observed Yields", zorder=3)
plt.plot(tau_fit, ns_fit, label="Nelson–Siegel Fit", linewidth=2)
plt.plot(tau_fit, spline_fit, label="Cubic Spline Fit", linestyle="--", linewidth=2)
plt.xlabel("Maturity (Years)")
plt.ylabel("Yield (%)")
plt.title("Yield Curve: Nelson–Siegel vs. Cubic Spline")
plt.legend()
plt.tight_layout()
plt.show()

print("Nelson–Siegel parameters:")
print(f"  β0 (level)    = {beta0:.4f}")
print(f"  β1 (slope)    = {beta1:.4f}")
print(f"  β2 (curvature)= {beta2:.4f}")
print(f"  λ  (decay)    = {lambd:.4f}")
